# Experiment with finding circles in the Rett Girls videos

July 2018

This attempts using boundaries and contained texture

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])

In [ ]:
def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2LAB)
    return im
show(grabFrame(100))

In [ ]:
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)

im = grabFrame(60)
pblue = isBlue(im)
_, tblue = cv2.threshold(pblue, 0.6, 255, cv2.THRESH_BINARY)
show(pblue)
show(tblue)
show(255 - tblue)

In [ ]:
def isCircular(contour):
    perimeter = cv2.arcLength(contour, True)
    if perimeter == 0:
        return False
    area = cv2.contourArea(contour)
    circularity = 4 * np.pi * (area / perimeter ** 2)
    #return 0.7 <= circularity <= 1.2
    return 0.5 <= circularity <= 1.4

def findBlobs(image, verbose=True):
    # find the special blue
    blue = isBlue(image)
    _, blue = cv2.threshold(blue, 0.6, 255, cv2.THRESH_BINARY)
    blue = blue.astype(np.uint8)
    # dilate a bit to fill in noise
    blue = cv2.dilate(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    blue = cv2.erode(blue, np.ones((3,3), dtype=np.uint8), iterations=1)
    
    if verbose:
        show(blue)
    # get the contours of the blue regions
    im2, contours, hierarchy = cv2.findContours(blue, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if verbose:
        oim = image.copy()
        cv2.drawContours(oim, contours, -1, (0,0,0), 1)
        show(oim)
    # filter by area
    if True:
        minArea = np.pi * 4**2
        maxArea = np.pi * 16**2
        contours = [contour for contour in contours if minArea < cv2.contourArea(contour) < maxArea]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)
    if False:
        # filter by circularity
        contours = [contour for contour in contours if isCircular(contour)]
        if verbose:
            oim = image.copy()
            cv2.drawContours(oim, contours, -1, (0,0,0), 1)
            show(oim)

    return contours
t = findBlobs(grabFrame(3680))

## Marc suggests using the digits

In [ ]:
f = grabFrame(60)
t = findBlobs(f)

In [ ]:
f = grabFrame(60)
digitContours = findBlobs(f, False)
b = isBlue(f)
digitOrder = [3, 4, 2, 9, 1, 7, 6, 0, 8, 5]
digitMap = {v:i for i,v in enumerate(digitOrder)}
offset = {
    1: (2,3),
    2: (2,3),
    3: (2,4),
    4: (2,3),
    5: (3,3),
    6: (2,3),
    7: (2,3),
    8: (1,4),
    9: (2,3),
    0: (2,6)   
}

for digit in range(0,10):
    c = digitContours[digitMap[digit]]
    x, y, w, h = cv2.boundingRect(c)
    #yo, xo = offset[digit]
    show(b[y:y+h,x:x+w], title='%d %d' % (x, y))


In [ ]:
def numberTemplate(number, scale):
    font = cv2.FONT_HERSHEY_COMPLEX_SMALL
    text = str(number)
    (w,h),b = cv2.getTextSize(text, font, scale, 1)
    img = np.zeros((h+b+4,w+4), dtype=np.uint8)
    cv2.putText(img, text, (2, h+b), font, scale, 255, 1, cv2.LINE_AA)
    x, y, w, h = cv2.boundingRect(cv2.findNonZero(img))
    return img[y:y+h,x:x+w]

show(numberTemplate(4, 0.57))

In [ ]:
f = grabFrame(608)
digitContours = findBlobs(f, False)
im = grabFrame(60)


b = isBlue(f)
_, b = cv2.threshold(b, 0.7, 255, cv2.THRESH_BINARY)
b = b.astype(np.uint8)
for c in digitContours[3:4]:
    x, y, w, h = cv2.boundingRect(c)
    print(w, h)
    area = (f[y:y+h,x:x+w, 0]).astype(np.uint8)
    show(area, title='%d %d' % (x, y))
    values = []
    scales = np.arange(0.3,0.7,0.01)
    for scale in scales:
        number = 2
        temp = numberTemplate(number, scale)
        res = cv2.matchTemplate(area, temp, cv2.TM_CCORR_NORMED)
        # res = cv2.matchTemplate(area, temp, cv2.TM_SQDIFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        print(scale, max_val, max_loc)
        values.append(max_val)
        # values.append(min_val)
    plt.figure()
    plt.plot(scales, values)
    plt.show()


In [ ]:
13*0.42/12

In [ ]:
f = grabFrame(2297)
show(f)
digitContours = findBlobs(f, False)
im = f

b = isBlue(f)
_, b = cv2.threshold(b, 0.7, 255, cv2.THRESH_BINARY)
b = b.astype(np.uint8)
for c in digitContours:
    x, y, w, h = cv2.boundingRect(c)
    area = (f[y:y+h,x:x+w, 0]).astype(np.uint8)
    show(area, title='%d %d' % (x, y))
    values = []
    scale = max(0.3, min(0.7, 0.42 * min(h, w) / 12))
    print(scale)
    for number in range(10):
        temp = numberTemplate(number, scale)
        try:
            res = cv2.matchTemplate(area, temp, cv2.TM_CCOEFF_NORMED)
        except:
            values.append(0)
        else:
            # res = cv2.matchTemplate(area, temp, cv2.TM_SQDIFF_NORMED)
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
            #print(scale, max_val, max_loc)
            values.append(max_val)
        # values.append(min_val)
    plt.figure()
    plt.plot(range(10), values)
    plt.show()

## Find all the blobs and save them away

In [ ]:
results = []
nframes = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
for fno in range(100, 3680):
    if fno % 100 == 0:
        print(fno, len(results))
    frame = grabFrame(fno)
    for blob in findBlobs(frame, False):
        x, y, w, h = cv2.boundingRect(blob)
        if x < 2 or y < 2 or x+w >= frame.shape[1] - 2 or y+h >= frame.shape[0] - 2:
            continue
        pixels = frame[y-2:y+h+2, x-2:x+w+2].copy()
        results.append((fno, blob, pixels))

    

In [ ]:
import pickle
pickle.dump(results, open('MSB_Video_1.blobs1', 'wb'))

## Eliminate duplicates

In [ ]:
compressed = []
for afno, ablob, apixels in results:
    if afno % 100 == 0:
        print(afno, len(compressed))
    for bfno, bblob, bpixels in compressed[::-1]:
        if len(ablob) == len(bblob) and np.all(ablob == bblob) and apixels.shape == bpixels.shape and np.all(apixels == bpixels):
            break
    else:
        compressed.append((afno, ablob, apixels))

In [ ]:
pickle.dump(compressed, open('MSB_Video_1_compressed.blobs', 'wb'))

## Eliminate near duplicates from previous frame only

In [ ]:
import itertools
compressed = []
previous = []
current = []
for af, ag in itertools.groupby(results, lambda x: x[0]):
    for a in ag:
        af, ab, ap = a
        ax, ay, aw, ah = cv2.boundingRect(ab)
        ac = np.array((ax+aw/2, ay+ah/2))
        ar = max(aw, ah)
        for b in previous:
            bf, bb, bp = b
            bx, by, bw, bh = cv2.boundingRect(bb)
            if len(ab) == len(bb) and np.all(ab == bb) and ap.size == bp.size and np.all(ap == bp):
                break # perfect dup
            bc = np.array((bx+bw/2, by+bh/2))
            br = max(bw, bh)
            if np.sqrt(np.sum((ac-bc)**2)) < 3 and abs(ar - br) < 3:
                break # near dup
        else:
            current.append(a)
    compressed.extend(previous)
    previous = current
    current = []
            
            
        

In [ ]:
len(compressed)